## PytorchZeroToAll. Lec07

In [1]:
from torch import nn, optim, from_numpy
import numpy as np
from google.colab import drive

drive.mount('/content/drive')
xy = np.loadtxt('/content/drive/My Drive/Colab Notebooks/data/diabetes.csv.gz', delimiter=',',dtype=np.float32)
x_data = from_numpy(xy[:, 0:-1])
y_data = from_numpy(xy[:, [-1]])
print(f"XW's shape: {x_data.shape} | YW's shape: {y_data.shape}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
XW's shape: torch.Size([759, 8]) | YW's shape: torch.Size([759, 1])


In [2]:
import torch

class Model(nn.Module):
  def __init__(self):
    """
    In the constructor we instantiate nn.Linear module
    """
    super(Model, self).__init__()
    self.l1 = nn.Linear(8,6)
    self.l2 = nn.Linear(6,4)
    self.l3 = nn.Linear(4,1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    """
    In the forward function we accept a Variable of input data and we must return
    a Variable of output data. We can use Modules defined in the constructor as 
    well as arbitary operators on Variables.
    """
    out1 = self.sigmoid(self.l1(x))
    out2 = self.sigmoid(self.l2(out1))
    y_pred = self.sigmoid(self.l3(out2))
    return y_pred

# our model
model = Model()

In [3]:
# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contaion the learnable parameters of the three
# nn.Linear modules which are members of the model.

criterion = torch.nn.BCELoss(reduction='mean')
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [4]:
# Training loop
for epoch in range(100):
  # Forward pass: Compute predicted y by passing x to the model
  y_pred = model(x_data)

  # Compute and print loss
  loss = criterion(y_pred, y_data)
  print(f'Epoch: {epoch+1}/1000 | Loss: {loss.item(): .4f}')
  
  # Zero gradients, perform a backward pass, and update the weights.
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

Epoch: 1/1000 | Loss:  0.6719
Epoch: 2/1000 | Loss:  0.6717
Epoch: 3/1000 | Loss:  0.6714
Epoch: 4/1000 | Loss:  0.6712
Epoch: 5/1000 | Loss:  0.6709
Epoch: 6/1000 | Loss:  0.6707
Epoch: 7/1000 | Loss:  0.6705
Epoch: 8/1000 | Loss:  0.6702
Epoch: 9/1000 | Loss:  0.6700
Epoch: 10/1000 | Loss:  0.6698
Epoch: 11/1000 | Loss:  0.6696
Epoch: 12/1000 | Loss:  0.6693
Epoch: 13/1000 | Loss:  0.6691
Epoch: 14/1000 | Loss:  0.6689
Epoch: 15/1000 | Loss:  0.6687
Epoch: 16/1000 | Loss:  0.6685
Epoch: 17/1000 | Loss:  0.6683
Epoch: 18/1000 | Loss:  0.6681
Epoch: 19/1000 | Loss:  0.6678
Epoch: 20/1000 | Loss:  0.6676
Epoch: 21/1000 | Loss:  0.6674
Epoch: 22/1000 | Loss:  0.6672
Epoch: 23/1000 | Loss:  0.6670
Epoch: 24/1000 | Loss:  0.6668
Epoch: 25/1000 | Loss:  0.6666
Epoch: 26/1000 | Loss:  0.6665
Epoch: 27/1000 | Loss:  0.6663
Epoch: 28/1000 | Loss:  0.6661
Epoch: 29/1000 | Loss:  0.6659
Epoch: 30/1000 | Loss:  0.6657
Epoch: 31/1000 | Loss:  0.6655
Epoch: 32/1000 | Loss:  0.6653
Epoch: 33/1000 | 

## PytorchZeroToAll. Lec08

In [5]:
from torch.utils.data import Dataset, DataLoader
from torch import from_numpy, tensor
import numpy as np

class DiabetesDataset(Dataset):
  """ Diabetes Dataset """

  # Initialize your data, download, etc.
  def __init__(self):
    xy = np.loadtxt('/content/drive/My Drive/Colab Notebooks/data/diabetes.csv.gz', delimiter=',',dtype=np.float32)
    self.len = xy.shape[0]
    self.x_data = from_numpy(xy[:, 0:-1])
    self.y_data = from_numpy(xy[:, [-1]])

  def __getitem__(self, index):
    return self.x_data[index], self.y_data[index]

  def __len__(self):
    return self.len

dataset = DiabetesDataset()

train_loader = DataLoader(dataset=dataset, 
                          batch_size=32,
                          shuffle=True, 
                          num_workers=2)

In [6]:
for epoch in range(2):
  for i, data in enumerate(train_loader, 0):
    # get the inputs
    inputs, labels = data

    # wrap them in Variable
    inputs, labels = tensor(inputs), tensor(labels)

    # Run your training process
    print(f'Epoch: {i} | Inputs {inputs.data} | Labels {labels.data}')

Epoch: 0 | Inputs tensor([[-0.5294,  0.1558,  0.1803,  0.0000,  0.0000, -0.1386, -0.7455, -0.1667],
        [-0.8824, -0.0452,  0.3443, -0.4949, -0.5745,  0.0432, -0.8676, -0.2667],
        [-0.1765,  0.0653, -0.0164, -0.5152,  0.0000, -0.2101, -0.8138, -0.7333],
        [-0.4118,  0.8995,  0.0492, -0.3333, -0.2317, -0.0700, -0.5687, -0.7333],
        [-0.0588,  0.7688,  0.4754, -0.3131, -0.2908,  0.0045, -0.6678,  0.2333],
        [-0.5294,  0.1156,  0.1803, -0.0505, -0.5106,  0.1058,  0.1204,  0.1667],
        [-0.1765,  0.9598,  0.1475, -0.3333, -0.6572, -0.2519, -0.9274,  0.1333],
        [-0.5294,  0.2965, -0.0164, -0.7576, -0.4539, -0.1803, -0.6166, -0.6667],
        [-0.7647,  0.4673,  0.0000,  0.0000,  0.0000, -0.1803, -0.8617, -0.7667],
        [ 0.0000, -0.0553,  0.1475, -0.4545, -0.7281,  0.2966, -0.7703,  0.0000],
        [-0.6471, -0.0352, -0.0820, -0.3131, -0.7281, -0.2638, -0.2605, -0.4000],
        [-0.8824, -0.2864,  0.2787,  0.0101, -0.8936, -0.0104, -0.7062,  0.0000]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


Epoch: 0 | Inputs tensor([[-0.8824,  0.1256,  0.1803, -0.3939, -0.5839,  0.0253, -0.6157, -0.8667],
        [-0.5294,  0.5678,  0.2295,  0.0000,  0.0000,  0.4396, -0.8634, -0.6333],
        [-0.6471,  0.7387,  0.2787, -0.2121, -0.5626,  0.0075, -0.2383, -0.6667],
        [-0.7647, -0.1256, -0.0492, -0.6768, -0.8771, -0.0253, -0.9249, -0.8667],
        [-0.6471,  0.8090,  0.0492, -0.4949, -0.8345,  0.0134, -0.8352, -0.8333],
        [-0.8824,  0.6884,  0.4426, -0.4141,  0.0000,  0.0432, -0.2938,  0.0333],
        [-0.4118,  0.0955,  0.2295, -0.4747,  0.0000,  0.0730, -0.6003,  0.3000],
        [-0.4118,  0.3266,  0.3115,  0.0000,  0.0000, -0.2012, -0.9078,  0.6000],
        [-0.8824,  0.4774,  0.5410, -0.1717,  0.0000,  0.4694, -0.7609, -0.8000],
        [-0.8824,  0.2261,  0.4754,  0.0303, -0.4799,  0.4814, -0.7891, -0.6667],
        [-0.6471,  0.9196,  0.1148, -0.6970, -0.6927, -0.0790, -0.8113, -0.5667],
        [-0.5294,  0.1055,  0.2459, -0.5960, -0.7636, -0.1535, -0.9658, -0.8000]

In [7]:
class Model(nn.Module):

  def __init__(self):
    """
    In the constructor we instantiate nn.Linear module
    """
    super(Model, self).__init__()
    self.l1 = nn.Linear(8,6)
    self.l2 = nn.Linear(6,4)
    self.l3 = nn.Linear(4,1)

    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    """
    In the forward function we accept a Variable of input data and we must return
    a Variable of output data. We can use Modules defined in the constructor as 
    well as arbitary operators on Variables.
    """
    out1 = self.sigmoid(self.l1(x))
    out2 = self.sigmoid(self.l2(out1))
    y_pred = self.sigmoid(self.l3(out2))

    return y_pred

# our model
model = Model()

In [8]:
# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contaion the learnable parameters of the three
# nn.Linear modules which are members of the model.

criterion = torch.nn.BCELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

# Training loop
for epoch in range(100):
  for i, data in enumerate(train_loader, 0): 
    # get the inputs
    inputs, labels = data

    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(inputs)

    # Compute and print loss
    loss = criterion(y_pred, labels)
    print(f'Epoch {epoch+1} | Batch: {i+1} | Loss: {loss.item(): .4f}')

    # Zero gradients, perform a backward pass, and update the weights
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Epoch 1 | Batch: 1 | Loss:  23.1261
Epoch 1 | Batch: 2 | Loss:  22.0861
Epoch 1 | Batch: 3 | Loss:  20.5000
Epoch 1 | Batch: 4 | Loss:  26.2923
Epoch 1 | Batch: 5 | Loss:  22.6727
Epoch 1 | Batch: 6 | Loss:  25.3414
Epoch 1 | Batch: 7 | Loss:  21.6732
Epoch 1 | Batch: 8 | Loss:  21.9570
Epoch 1 | Batch: 9 | Loss:  20.6208
Epoch 1 | Batch: 10 | Loss:  19.9732
Epoch 1 | Batch: 11 | Loss:  17.3877
Epoch 1 | Batch: 12 | Loss:  23.4270
Epoch 1 | Batch: 13 | Loss:  23.0617
Epoch 1 | Batch: 14 | Loss:  22.2084
Epoch 1 | Batch: 15 | Loss:  23.0267
Epoch 1 | Batch: 16 | Loss:  22.2321
Epoch 1 | Batch: 17 | Loss:  21.4538
Epoch 1 | Batch: 18 | Loss:  20.2430
Epoch 1 | Batch: 19 | Loss:  20.6934
Epoch 1 | Batch: 20 | Loss:  20.5848
Epoch 1 | Batch: 21 | Loss:  20.6077
Epoch 1 | Batch: 22 | Loss:  23.1805
Epoch 1 | Batch: 23 | Loss:  22.0665
Epoch 1 | Batch: 24 | Loss:  14.8799
Epoch 2 | Batch: 1 | Loss:  19.2869
Epoch 2 | Batch: 2 | Loss:  20.9371
Epoch 2 | Batch: 3 | Loss:  21.2088
Epoch 2 | Bat

## PytorchZeroToAll. Lec09

In [9]:
from __future__ import print_function
from torch import nn, optim, cuda
from torch.utils import data
from torchvision import datasets, transforms
import torch.nn.functional as F
import time

In [10]:
batch_size = 64
device = 'cuda' if cuda.is_available() else 'cpu'
print(f'Training MNIST Model on {device}\n{"=" * 44}')

# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = data.DataLoader(dataset=train_dataset, 
                               batch_size=batch_size, 
                               shuffle=True)

test_loader = data.DataLoader(dataset=test_dataset, 
                              batch_size=batch_size, 
                              shuffle=False)

Training MNIST Model on cuda


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [11]:
class Net(nn.Module):

  def __init__(self):
    super(Net, self).__init__()
    self.l1 = nn.Linear(784, 520)
    self.l2 = nn.Linear(520, 320)
    self.l3 = nn.Linear(320, 240)
    self.l4 = nn.Linear(240, 120)
    self.l5 = nn.Linear(120, 10)

  def forward(self, x):
    # Flatten the data (n, 1, 28, 28) -> (n, 784)
    x = x.view(-1, 784)
    x = F.relu(self.l1(x))
    x = F.relu(self.l2(x))
    x = F.relu(self.l3(x))
    x = F.relu(self.l4(x))
    return self.l5(x) # No need activation

model = Net()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [12]:
def train(epoch):
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output, target) 
    loss.backward()
    optimizer.step()
    if batch_idx % 10 == 0:
      print('Train Epoch: {} | Batch Status: {}/{} ({:.0f}%) | Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [13]:
def test():
  model.eval()
  test_loss = 0
  correct = 0
  for data, target in test_loader:
    data, target = data.to(device), target.to(device)
    output = model(data)
    # sum up batch loss
    test_loss += criterion(output, target).item()
    # get the index of the max
    pred = output.data.max(1, keepdim=True)[1]
    correct += pred.eq(target.data.view_as(pred)).cpu().sum()

  test_loss /= len(test_loader.dataset)
  print(f'===========================\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)')

In [14]:
if __name__ == '__main__':
    since = time.time()
    for epoch in range(1,10):
      epoch_start = time.time()
      train(epoch)
      m, s = divmod(time.time() - epoch_start, 60)
      print(f'Training time: {m:.0f}m {s:.0f}s')
      test()
      m, s = divmod(time.time() - epoch_start, 60)
      print(f'Testing time: {m:.0f}m {s:.0f}s')

    m, s = divmod(time.time() - since, 60)
    print(f'Total Time: {m:.0f}m {s:.0f}s\nModel was trained on {device}!')

Train Epoch: 1 | Batch Status: 0/60000 (0%) | Loss: 2.308764
Train Epoch: 1 | Batch Status: 640/60000 (1%) | Loss: 2.304667
Train Epoch: 1 | Batch Status: 1280/60000 (2%) | Loss: 2.303483
Train Epoch: 1 | Batch Status: 1920/60000 (3%) | Loss: 2.298565
Train Epoch: 1 | Batch Status: 2560/60000 (4%) | Loss: 2.304631
Train Epoch: 1 | Batch Status: 3200/60000 (5%) | Loss: 2.307826
Train Epoch: 1 | Batch Status: 3840/60000 (6%) | Loss: 2.305692
Train Epoch: 1 | Batch Status: 4480/60000 (7%) | Loss: 2.296519
Train Epoch: 1 | Batch Status: 5120/60000 (9%) | Loss: 2.313386
Train Epoch: 1 | Batch Status: 5760/60000 (10%) | Loss: 2.303475
Train Epoch: 1 | Batch Status: 6400/60000 (11%) | Loss: 2.303573
Train Epoch: 1 | Batch Status: 7040/60000 (12%) | Loss: 2.303077
Train Epoch: 1 | Batch Status: 7680/60000 (13%) | Loss: 2.297778
Train Epoch: 1 | Batch Status: 8320/60000 (14%) | Loss: 2.287597
Train Epoch: 1 | Batch Status: 8960/60000 (15%) | Loss: 2.288209
Train Epoch: 1 | Batch Status: 9600/60

KeyboardInterrupt: ignored